In [4]:
import sys, os
sys.path.append(os.path.abspath(os.path.join(os.getcwd(), '..')))

from config import LensConfig
from Data_Downloader.coingecko import CoinGeckoDownloader
from Data_Downloader.binance_ccxt import BinanceDownloader
from Data_Downloader.data_manager import DataManager

In [5]:
# --- Load config ---
cfg = LensConfig()

# --- CoinGecko ---
cg = CoinGeckoDownloader(cfg)
universe = cg.fetch_top_universe(cfg.cg_top_n)
cg_files = cg.fetch_and_save_all()

# --- Filter and map symbols ---
cfg.binance_symbols = cg.universe_to_binance_symbols(universe)
print(f"Filtered Binance symbols: {cfg.binance_symbols}")

# --- Binance ---
binance = BinanceDownloader(cfg)
binance_files = binance.fetch_and_save()

⏭️ Skipping Bitcoin (btc) – already exists.
⏭️ Skipping Ethereum (eth) – already exists.
⏭️ Skipping XRP (xrp) – already exists.
⏭️ Skipping Tether (usdt) – already exists.
⏭️ Skipping BNB (bnb) – already exists.
⏭️ Skipping Solana (sol) – already exists.
⏭️ Skipping USDC (usdc) – already exists.
⏭️ Skipping Dogecoin (doge) – already exists.
⏭️ Skipping Lido Staked Ether (steth) – already exists.
⏭️ Skipping TRON (trx) – already exists.
⏭️ Skipping Cardano (ada) – already exists.
⏭️ Skipping Wrapped stETH (wsteth) – already exists.
⏭️ Skipping Wrapped Beacon ETH (wbeth) – already exists.
⏭️ Skipping Chainlink (link) – already exists.
⏭️ Skipping Wrapped Bitcoin (wbtc) – already exists.
⏭️ Skipping Ethena USDe (usde) – already exists.
⏭️ Skipping Figure Heloc (figr_heloc) – already exists.
⏭️ Skipping Avalanche (avax) – already exists.
⏭️ Skipping Hyperliquid (hype) – already exists.
⏭️ Skipping Sui (sui) – already exists.
⏭️ Skipping Stellar (xlm) – already exists.
⏭️ Skipping Bitcoin 

Merging :

In [14]:
import pandas as pd

print("CG file sample:", cg_files[0])
print(pd.read_csv(cg_files[0], nrows=5).columns)

print("BN file sample:", bn_files[0])
print(pd.read_csv(bn_files[0], nrows=5).columns)


CG file sample: data/data_Last_4_Years\coingecko\0g_zero-gravity.csv
Index(['date', 'price', 'market_cap', 'volume', 'circulating_supply',
       'total_supply', 'max_supply', 'market_cap_rank', 'symbol'],
      dtype='object')
BN file sample: data/data_Last_4_Years\binance\ohlcv_2024-11.csv
Index(['rank', 'symbol', 'name', 'exchange', 'exchange_symbol', 'time', 'open',
       'high', 'low', 'close', 'volume', 'month_start', 'snapshot_date'],
      dtype='object')


In [15]:
# Standardize columns before merging
if "time" in bn_df.columns:
    bn_df = bn_df.rename(columns={"time": "date"})
elif "snapshot_date" in bn_df.columns:
    bn_df = bn_df.rename(columns={"snapshot_date": "date"})

# Make sure it's datetime
bn_df["date"] = pd.to_datetime(bn_df["date"]).dt.date
cg_df["date"] = pd.to_datetime(cg_df["date"]).dt.date


NameError: name 'bn_df' is not defined

In [ ]:
import os, glob
from Data_Downloader.data_manager import DataManager

# Paths
BASE_DIR = "data/data_Last_4_Years"
CG_DIR = os.path.join(BASE_DIR, "coingecko")
BN_DIR = os.path.join(BASE_DIR, "binance")

cg_files = glob.glob(os.path.join(CG_DIR, "*.csv"))
bn_files = glob.glob(os.path.join(BN_DIR, "*.csv"))

manager = DataManager(outdir=BASE_DIR)
merged = manager.merge_sources(cg_files, bn_files, save=True, filename="merged_sources.csv")

print("✅ Merged dataset shape:", merged.shape)
display(merged.head())



In [12]:
import plotly.graph_objects as go

# Count non-null values
cg_cols = [c for c in merged.columns if c.endswith("_cg")]
bn_cols = [c for c in merged.columns if c.endswith("_binance")]

cg_count = merged[cg_cols].notna().sum().sum()
bn_count = merged[bn_cols].notna().sum().sum()

# Build comparison bar chart
fig = go.Figure()
fig.add_trace(go.Bar(x=["CoinGecko"], y=[cg_count], name="CoinGecko"))
fig.add_trace(go.Bar(x=["Binance"], y=[bn_count], name="Binance"))

fig.update_layout(
    title="Data Availability: Binance vs CoinGecko",
    yaxis_title="Non-missing values",
    barmode="group",
)
fig.show()


NameError: name 'merged' is not defined

In [13]:
import pandas as pd
import plotly.express as px
import plotly.graph_objects as go

# --- Load merged data ---
merged = pd.read_csv("data/data_Last_4_Years/merged_sources.csv", parse_dates=["date"])

# --- 1. Distribution of data available per month ---
# Count how many symbols have data each month (for any field, e.g. price_cg)
monthly_counts = (
    merged.dropna(subset=["price_cg"])  # use CoinGecko price as baseline
    .groupby([merged["date"].dt.to_period("M")])["symbol"]
    .nunique()
    .reset_index()
)
monthly_counts["date"] = monthly_counts["date"].dt.to_timestamp()

fig1 = px.bar(
    monthly_counts,
    x="date",
    y="symbol",
    title="Distribution of Available Data (Monthly, CoinGecko)",
    labels={"symbol": "Number of Coins", "date": "Month"},
)
fig1.show()

FileNotFoundError: [Errno 2] No such file or directory: 'data/data_Last_4_Years/merged_sources.csv'

In [ ]:
import pandas as pd
import plotly.express as px

# --- Load merged dataset ---
merged = pd.read_csv("data/data_Last_4_Years/merged_sources.csv", parse_dates=["date"])

# --- Select Binance OHLCV fields ---
binance_cols = ["open_binance", "high_binance", "low_binance", "close_binance", "volume_binance"]

# Count non-null values for each Binance field
availability = merged[binance_cols].notna().sum().reset_index()
availability.columns = ["field", "count"]

# Normalize to percentage of total rows
availability["percent"] = 100 * availability["count"] / len(merged)

# --- Plot ---
fig = px.bar(
    availability,
    x="field",
    y="percent",
    text=availability["percent"].round(1).astype(str) + "%",
    title="Availability of Binance OHLCV Fields in Dataset",
    labels={"percent": "Coverage (%)", "field": "Binance Field"},
    color="percent",
    color_continuous_scale="Blues",
)

fig.update_traces(textposition="outside")
fig.update_layout(yaxis=dict(range=[0, 100]))
fig.show()


: 

: 